# Gemma 모델로 텍스트 요약하기 (Hugging Face + LangChain)

1. LLM 모델 로드 (pipeline, chatopenapi)
2. Prompt 템플릿 생성
3. Chains 으로 프롬프트와 모델을 연결
5. Query 입력을 모델에 입력
6. 결과를 postprocessing

In [32]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
# Hugging Face에서 Gemma 모델 불러오기
model_id = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16,    # 메모리 절약용 float 타입
    device_map='auto'    # GPU 자동 할당
)

Fetching 4 files:   0%|          | 0/4 [11:23<?, ?it/s]


In [ ]:
# Hugging Face 파이프라인 구성 (text generation)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512, # 생성 최대 길이
    temperature=0.0,    # 결정적 출력 (창의성 ↓, 정확성 ↑)
    do_sample=False     # 무작위 샘플링 안 함
)

In [ ]:
# LangChain에서 사용할 수 있게 LLM wraping
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# 프롬프트 템플릿 작성 (요약 지시 포함)
summary_template = """
당신은 전문 요약가입니다.
아래 주어진 텍스트를 50단어 이내로 간결하게 요약해 주세요.

텍스트:
{text_to_summarize}

요약:
"""
summary_prompt = PromptTemplate.from_template(summary_template)

In [ ]:
# LLMChain으로 요약 체인 구성
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, verbose=True)

In [ ]:
# 요약할 실제 텍스트 입력
text_to_summarize = """\
[데일리안 = 조인영 기자] 우리나라 앱 개발자가 앱 마켓사업자에게서 경험하는 주요 불공정 사례는 심사 지연과 등록 거부이며, 앱 내 결제(인앱결제)의 가장 큰 문제점은 ‘과도한 수수료’라고 인식하고 있는 것으로 나타났다.

방송통신위원회와 한국인터넷진흥원은 11일 '전기통신사업법' 제22조의9(앱 마켓사업자의 의무 및 실태조사)에 따른 ‘2024년도 앱 마켓 실태조사 결과’를 발표했다.

2023년도 국내 ‘앱 마켓’ 규모는 거래액 기준으로 8조1952억원으로 2022년 8조 7598억원 대비 6.4% 감소했다

각 사업자별로 보면, 애플 앱스토어(10.1%)와 삼성전자 갤럭시스토어(6.3%)는 전년 대비 매출이 증가했으며, 구글 플레이(△10.1%)와 원스토어(△21.6%)는 감소했다.

4개 앱 마켓사업자의 거래액 대비 수수료 비중은 약 14~26% 수준이며, 앱 등록 매출의 경우 애플 앱스토어는 약 9.4% 증가했고 구글 플레이는 약 12.9% 감소했다.

또한 국내 앱 마켓에 등록된 전체 앱 수는 전년 대비 0.1% 증가한 531만8182개(각 앱 마켓별 중복 포함), 앱 개발자 수는 전년 대비 0.65% 적은 163만6655명(각 앱 마켓별 중복 포함)으로 나타났다.

분야별 앱 등록 비중은 사진‧도구(26.1%), 라이프스타일(15.6%), 미디어‧출판(14.5%) 관련 앱이 전체의 56.2%를 차지했다.

국내 앱 개발자가 이용하는 앱 마켓은 구글 플레이(96.4%), 애플 앱스토어(71.3%) 순(중복포함)이며, 매출액 비중도 구글 플레이가 67.5%로 가장 높았다. 그 를 애플 앱스토어(28.2%), 원스토어(2.9%), 갤럭시스토어(1.5%)가 따랐다.

앱 개발자가 느끼는 주요 불공정 사례로는 앱 심사지연 경험(애플 앱스토어 6.8%, 구글 플레이 26.2%)이 가장 높았으며, 앱 등록 거부 경험(애플 20%, 구글 3%)과 앱 삭제 경험(구글 8.2%, 애플 3.2%)이 그 뒤를 이었다.

앱 내 결제의 가장 큰 문제점은 ‘과도한 수수료’라고 답한 앱 개발자가 0.4%로 가장 높게 나타났으며, 다음으로 ‘환불 등 수익 정산의 불명확함’(11.6%), ‘결제 수단 선택 제한’(8.9%) 순으로 조사됐다.

앱 개발자가 느낀 앱 최초 등록 과정상의 어려움으로는 ‘심사 기준이 확하지 않음’(구글 플레이 29.8%, 애플 앱스토어 29.6%), ‘질의에 대한 드백 지연’(각각 26.1%, 25.3%) 등이 꼽혔다.

앱을 최초로 등록하기 위해 소요되는 심사기간은 구글 플레이는 등록 시 일 이내(25.6%)에 처리되는 경우가 많았으나, 애플 앱스토어는 6∼7일 이내(42.5%)로 나타났다.

우리나라 국민이 가장 자주 이용하는 앱 마켓은 구글 플레이(67.2%), 애플 스토어(29.7%) 순이었으며, 해당 앱 마켓을 자주 이용하는 이유로는 ‘사용이 편리해서’(67.7%), ‘설치되어 있어서’(61.3%), ‘상품 수가 많아서’(33,5%) 등 으로 나타났다.
"""


In [ ]:
# 요약 실행
summary_result = summary_chain.run(text_to_summarize)
print("요약 결과:", summary_result)

# GPT-4o-mini로 상황에 맞는 모델 추천받기 (ChatOpenAI)

In [ ]:
# OpenAI 모델(GPT-4o-mini) 사용
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
# LLM 정의 (GPT-4o-mini, 창의성 없는 결정적 출력)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [ ]:
# 대화형 프롬프트 구성 (system + user 역할 분리)
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문의 상황에 적절한 인공지능 모델을 찾아주는 AI 전문가입니다."),
    ("user", "다음 상황에 맞는 모델을 찾아줘 \n{input}")
])

In [ ]:
# LLMChain 구성
chat_chain = LLMChain(llm=llm, prompt=chat_template, verbose=True)

In [ ]:
# 사용자 질문 입력 및 실행
user_input = "우리는 CCTV에 범죄 상황을 인지하기 위한 기능을 넣으려고 합니다."
response = chat_chain.run(input=user_input)

In [ ]:
print("GPT 추천 결과:", response)

# 날짜 계산 Agent (Tool 1개)
: LangChain의 Agent 기능을 활용해서 도구(tool)를 사용하는 지능형 에이전트를 만드는 실습 예제

:특히, 내일 날짜를 알려줘라는 요청에 대해, 모델이 직접 계산하지 않고, **직접 정의한 함수(get_tomorrow_data)** 를 호출해서 답하는 구조

#### 전체 코드 흐름
① GPT 모델 정의 (ChatOpenAI)

② 시스템 프롬프트 정의

③ 날짜 계산 함수 정의 (Tool 등록)

④ Tool 기반 Agent 생성

⑤ 사용자 입력 처리 및 함수 자동 호출

⑥ 결과 출력


In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool, create_react_agent
from datetime import datetime, timedelta
from langchain import hub

In [ ]:
# 1. 모델 불러오기
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0.0)

: temperature=0.0: 가장 안정적인 응답 유도 (창의성 ↓, 정확성 ↑)

In [ ]:
# 2. 에이전트용 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 다양한 도구를 활용해서 질문에 답변할 수 있습니다."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

- placeholder : 에이전트 내부에서 사용하는 scratchpad (도구 호출 관련 기록 저장)

In [6]:
prompt

ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessag

In [ ]:
# 3. tool 함수 정의
@tool
# 내일 날짜를 반환하는 간단한 함수
def get_tomorrow_data() -> str:
    """ return tomorrow datetime in format of %Y-%m-%d """
    return (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

- @tool 데코레이터 : 해당 함수를 LangChain에서 에이전트가 호출 가능한 도구로 등록

In [ ]:
# 4. 도구 리스트 등록
# 여러 개의 도구를 등록할 수 있다. (여기선 1개)
tools = [get_tomorrow_data]

In [ ]:
# 5. Agent 생성
# create_tool_calling_agent() : 
# 이 에이전트는 질문을 이해하고 적절한 도구를 고른 후 자동 실행
agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt)

In [ ]:
# 6. AgentExecutor로 실행기 구성
# AgentExecutor : 
# 실제로 사용자의 입력을 받고 응답을 처리하는 실행 엔진
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose=True,   # 실행 과정 자세히 출력
    handle_parsing_errors = True    # 오류 발생 시 자동 처리
    )

In [ ]:
# 7. 사용자 입력 → 에이전트 실행
# invoke()를 사용해 입력을 전달하고 결과를 받음.
result = agent_executor.invoke({"input": "내일의 날짜를 알려줘."})
print(result)



> Entering new AgentExecutor chain...

Invoking: `get_tomorrow_data` with `{}`


2025-07-17내일의 날짜는 2025년 7월 17일입니다.

> Finished chain.
{'input': '내일의 날짜를 알려줘.', 'output': '내일의 날짜는 2025년 7월 17일입니다.'}


# LangChain Agents, Tool 2개
**두 함수를 모두 사용해 하나의 응답 만들기**
- get_tomorrow() → 내일 날짜 반환
- get_current_time() → 현재 시각 반환

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import tool, create_tool_calling_agent, AgentExecutor

from datetime import datetime, timedelta

In [5]:
# 1. LLM 정의 (GPT-4o-mini 사용)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [6]:
# 2. Prompt 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 도구를 활용해 사용자 질문에 답변하는 똑똑한 AI입니다."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [8]:
# 3. 첫 번째 도구: 내일 날짜 반환
@tool
def get_tomorrow() -> str:
    """내일 날짜를 YYYY-MM-DD 형식으로 반환합니다."""
    return (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

In [9]:
# 4. 두 번째 도구: 현재 시간 반환
@tool
def get_current_time() -> str:
    """현재 시간을 HH:MM:SS 형식으로 반환합니다."""
    return datetime.now().strftime("%H:%M:%S")

In [10]:
# 5. 도구 목록 정의
tools = [get_tomorrow, get_current_time]

In [11]:
# 6. 에이전트 생성
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [12]:
# 7. 실행기 설정
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [13]:
# 8. 사용자 입력
query = "내일 날짜랑 지금 시간을 알려줘."

In [14]:
# 9. 실행
result = agent_executor.invoke({"input": query})
print("\n 답변:\n", result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_tomorrow` with `{}`


2025-07-17
Invoking: `get_current_time` with `{}`


16:27:02내일 날짜는 2025-07-17이고, 현재 시간은 16:27:02입니다.

> Finished chain.

 답변:
 내일 날짜는 2025-07-17이고, 현재 시간은 16:27:02입니다.


# LangChain에서 ReAct 에이전트 구조를 활용
사용자의 자연어 질문에 대해 도구(tool)를 순차적으로 선택·실행하면서 답을 만들어내는 예제

<흐름>

Question: 내일의 날씨를 알려줘
<br>Thought: 내일 날짜를 알아야 한다
<br>Action: get_tomorrow_data
<br>Action Input: 없음
<br>Observation: 2025-07-17

Thought: 해당 날짜의 날씨를 알아야 한다
<br>Action: get_weather
<br>Action Input: 2025-07-17
<br>Observation: 맑음

Thought: 이제 최종 답을 알고 있습니다
<br>Final Answer: 내일(2025-07-17)의 날씨는 맑음입니다.

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import tool, create_react_agent, AgentExecutor
from datetime import datetime, timedelta

In [7]:
# 1. LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [8]:
# 2. 도구(tool) 정의
@tool
def get_tomorrow_data() -> str:
    """내일 날짜를 YYYY-MM-DD 형식으로 반환합니다."""
    return (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

@tool
def get_weather(date: str) -> str:
    """지정한 날짜의 날씨를 반환합니다. (예시: 항상 '맑음')"""
    return '맑음'

tools = [get_tomorrow_data, get_weather]

In [9]:
# 3. ReAct 형식의 프롬프트 템플릿
template = """ 다음 질문들에 최선을 다해서 답해주세요. 다음 도구들을 사용할 수 있습니다:

{tools}

다음 형식을 사용하세요:

Question: 답해야 할 입력 질문
Thought : 무엇을 해야 할지 항상 고민하세요
Action : 수행할 작업, [{tool_names}] 중 하나여야 합니다
Action Input : 작업에 대한 입력
Observation: 작업의 결과
... (위의 Thought/Action/Action Input/Observation 단계는 N 번 반복될 수 있습니다)
Thought : 이제 최종 답을 알고 있습니다
Final Answer: 원래 질문에 대한 최종 답변

시작!

Question: {input}
Though: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

In [10]:
# 4. ReAct 에이전트 생성
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [11]:
# 5. 에이전트 실행기 설정
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # 내부 Thought/Action 흐름 출력
)

In [12]:
# 6. 사용자 입력 실행
query = "내일의 날씨를 알려줘."
result = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
내일의 날짜를 알아야 내일의 날씨를 확인할 수 있습니다.  
Action: get_tomorrow_data  
Action Input: 없음  
2025-07-17이제 내일의 날짜를 알았으니, 그 날짜의 날씨를 확인해야 합니다.  
Action: get_weather  
Action Input: 2025-07-17  
맑음이제 최종 답을 알고 있습니다.  
Final Answer: 내일의 날씨는 맑음입니다.

> Finished chain.


In [13]:
# 7. 결과 출력
print("\n답변:\n", result["output"])


답변:
 내일의 날씨는 맑음입니다.


# 웹 검색 도구 - DuckDuckGoSearchResults
LangChain ReAct 기반 에이전트에 DuckDuckGo 웹 검색 도구를 연결

<실행 흐름>
- Question: 최근 멀티헤드 어텐션을 변형한 아키텍처를 찾아주세요
- Thought: 최신 기술 동향을 찾기 위해 검색이 필요합니다.
- Action: DuckDuckGoSearchResults
- Action Input: recent variants of multi-head attention
- Observation: ...검색 결과 목록...
- Thought: 정보를 바탕으로 답변할 수 있습니다.
- Final Answer: 최근 변형된 어텐션 아키텍처로는 Linformer, Performer, Longformer 등이 있습니다.


In [14]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent, tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain import hub

In [15]:
# 1. LLM 설정 (gpt-4o-mini 사용)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [16]:
# 2. 프롬프트 템플릿 (ReAct 스타일) 가져오기
prompt = hub.pull("hwchase17/react")

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [20]:
# 3. 검색 도구 정의 (DuckDuckGo)
search_tool = DuckDuckGoSearchResults()
tools = [search_tool]

In [21]:
# 4. ReAct 에이전트 생성
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [22]:
# 5. 에이전트 실행기 구성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,                # 사고과정 출력
    handle_parsing_errors=True  # 응답 파싱 오류 자동 처리
)

In [23]:
# 6. 사용자 질문 실행
query = "최근 멀티헤드 어텐션을 변형한 아키텍처를 찾아주세요"
result = agent_executor.invoke({"input": query})




> Entering new AgentExecutor chain...
최근 멀티헤드 어텐션을 변형한 아키텍처에 대한 정보를 찾기 위해 검색해보겠습니다.  
Action: duckduckgo_results_json  
Action Input: "recent architectures modified multi-head attention"  

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: Jan 22, 2025 · This study demonstrates that CNNs and ViT architectures stem from a unified underlying learning mechanism, which quantitatively measures the single-nodal performance …, title: Unified CNNs and transformers underlying learning mechanism …, link: https://arxiv.org/abs/2501.12900, snippet: 1 day ago · This paper presents a new architecture for multimodal sentiment analysis exploiting hierarchical cross-modal attention mechanisms, as well as two parallel lanes for audio analysis., title: Hierarchical cross-modal attention and dual audio pathways for …, link: https://www.nature.com/articles/s41598-025-09000-3, snippet: Jun 5, 2025 · The Multi-DConv-Head Attention (MDHA) addresses this limitation by integrating depthwise convolutions within the multi-head attention framework. This architectural innovation …, title: Multi-DConv-Head Attention: A Deep Learning Inovation for …, link: https://serp.ai/posts/multi-dconv-head-attention/, snippet: Feb 13, 2025 · Multi-head at

In [24]:
# 7. 결과 출력
print("\n답변:\n", result["output"])


답변:
 최근 멀티헤드 어텐션을 변형한 아키텍처로는 "Multi-DConv-Head Attention"이 있으며, 이는 깊이별 합성곱을 통합하여 성능을 향상시키는 혁신적인 접근법입니다.


# 논문 검색 도구 - Arxiv
사용자가 묻는 최신 연구 트렌드(예: 멀티헤드 어텐션 변형)에 대해 실제 논문 검색을 통해 응답

In [25]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
    load_tools,
)

In [26]:
# 1. LLM 설정 (ChatGPT 기반, 창의성 있는 답변 유도)
llm = ChatOpenAI(temperature=0.7)

In [27]:
# 2. ReAct 프롬프트 템플릿 불러오기
prompt = hub.pull("hwchase17/react")

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
# 3. Arxiv 검색 도구 로드
tools = load_tools(["arxiv"])

In [29]:
# 4. ReAct 기반 에이전트 생성
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [30]:
# 5. 에이전트 실행기 설정
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,                # 사고 과정 출력
    handle_parsing_errors=True  # 파싱 오류 자동 처리
)

In [31]:
# 6. 사용자 질문 실행
query = "2023년 이후로 나온 멀티헤드어텐션의 변형 모델들의 종류는?"
result = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I should search for recent papers on variations of multi-head attention models in NLP.
Action: arxiv
Action Input: multi-head attention NLP variations after 2023

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langchain_community/utilities/arxiv.py:103: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  return self.arxiv_search(


Published: 2024-08-02
Title: Should We Attend More or Less? Modulating Attention for Fairness
Authors: Abdelrahman Zayed, Goncalo Mordido, Samira Shabanian, Sarath Chandar
Summary: The advances in natural language processing (NLP) pose both opportunities and
challenges. While recent progress enables the development of high-performing
models for a variety of tasks, it also poses the risk of models learning
harmful biases from the data, such as gender stereotypes. In this work, we
investigate the role of attention, a widely-used technique in current
state-of-the-art NLP models, in the propagation of social biases. Specifically,
we study the relationship between the entropy of the attention distribution and
the model's performance and fairness. We then propose a novel method for
modulating attention weights to improve model fairness after training. Since
our method is only applied post-training and pre-inference, it is an
intra-processing method and is, therefore, less computationally exp

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langchain_community/utilities/arxiv.py:103: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  return self.arxiv_search(


I have found several recent papers on variations of multi-head attention models in NLP, but none specifically mention models that have come after 2023. I should broaden my search to include all relevant papers on multi-head attention models in NLP.
Action: arxiv
Action Input: multi-head attention NLP variations

...

I have found some recent papers on variations of multi-head attention models in NLP, including a quantum-enhanced attention mechanism and a model for Hausa NLP. There are no specific models mentioned that have come after 2023. I will search for more recent papers to find the complete list of models.
Action: arxiv
Action Input: multi-head attention NLP variationsPublished: 2025-06-27
Title: Quantum-Enhanced Attention Mechanism in NLP: A Hybrid Classical-Quantum Approach
Authors: S. M. Yousuf Iqbal Tomal, Abdullah Al Shafin, Debojit Bhattacharjee, MD. Khairul Amin, Rafiad Sadat Shahir
Summary: Recent advances in quantum computing have opened new pathways for enhancing
deep learning architectures, particularly in domains characterized by
high-dimensional and context-rich data such as natural language processing
(NLP). In this work, we present a hybrid classical-quantum Transformer model
that integrates a quantum-enhanced attention mechanism into the standard
classical architecture. By emb

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langchain_community/utilities/arxiv.py:103: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  return self.arxiv_search(


I have found some recent papers on variations of multi-head attention models in NLP, including a quantum-enhanced attention mechanism and a model for Hausa NLP. There are no specific models mentioned that have come after 2023. I will search for more recent papers to find the complete list of models.
Action: arxiv
Action Input: multi-head attention NLP variationsPublished: 2025-06-27
Title: Quantum-Enhanced Attention Mechanism in NLP: A Hybrid Classical-Quantum Approach
Authors: S. M. Yousuf Iqbal Tomal, Abdullah Al Shafin, Debojit Bhattacharjee, MD. Khairul Amin, Rafiad Sadat Shahir
Summary: Recent advances in quantum computing have opened new pathways for enhancing
deep learning architectures, particularly in domains characterized by
high-dimensional and context-rich data such as natural language processing
(NLP). In this work, we present a hybrid classical-quantum Transformer model
that integrates a quantum-enhanced attention mechanism into the standard
classical architecture. By emb

/opt/anaconda3/envs/J_Note/lib/python3.9/site-packages/langchain_community/utilities/arxiv.py:103: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  return self.arxiv_search(


After searching through multiple papers on variations of multi-head attention models in NLP, including recent ones, I was unable to find specific models that have come out after 2023. The models mentioned are either from 2023 or 2024. It seems that there may not be many papers specifically mentioning models that have come out after 2023. I will summarize the information I have gathered so far.
Final Answer: There are variations of multi-head attention models in NLP, such as quantum-enhanced attention mechanisms and models for specific languages like Hausa NLP, but there are no specific models mentioned that have come out after 2023 in the papers searched in arxiv.

> Finished chain.


In [32]:
# 7. 결과 출력
print("\n답변:\n", result["output"])


답변:
 There are variations of multi-head attention models in NLP, such as quantum-enhanced attention mechanisms and models for specific languages like Hausa NLP, but there are no specific models mentioned that have come out after 2023 in the papers searched in arxiv.


<흐름>
- Question: 2023년 이후로 나온 멀티헤드어텐션의 변형 모델들의 종류는?

- Thought: 최신 논문에서 관련 모델을 찾아야 합니다.
- Action: arxiv
- Action Input: multi-head attention variants after 2023
- Observation: 논문 목록 반환 (Linformer++, Hybrid MHA 등...)

- Thought: 논문들을 참고해서 모델 이름을 정리합니다.
Final Answer: 2023년 이후 등장한 변형 모델에는 Linformer++, Hybrid Global-Local Attention 등이 있습니다.